**Official implementation of paper "Revisiting In-context Learning Inference Circuit in Large Language Models" (ICLR 2025)**:

### Ablation Study

This experiment is to ablate the proposed ICL inference circuit and observe the performance changes, results are in Table 1, 2.

Author: Hakaze Cho, yfzhao@jaist.ac.jp, 2024/09

Organized, commented, and modified by: Hakaze Cho, 2025/02/03

**Part I: Import, Define, and Load Everything**

What you should do:
1. [Cell 1] Change to the path from your working directory to the directory containing the README.md file.
2. [Cell 2] Define your experiment parameters.
3. Run the Cell 1 - 4.

In [1]:
# Cell 1: Import libraries and change the working directory.

## Change the working directory
import os
try:
    # Change to the path from your working directory to the directory containing the README.md file.
    os.chdir("ICL_Inference_Dynamics_Released") 
except:
    print("Already in the correct directory or the directory does not exist.")

## Import libraries
from util import load_model_and_data, ablation_study
import StaICC

About the parameter `ablation_type`:

Alternative: 

- `"default_causal_attention_mask"`: no ablation (line 1 and 6 (given `k = 0`) in Table 1) 
- `"xi_to_si"`: disconnect the attention link from every token in the demonstrations to its forerunner token (line 2 in Table 1)
- `"xq_to_sq"`: disconnect the attention link from every token in the query to its forerunner token (line 3 in Table 1)
- `"si_to_yi"`: disconnect the attention link from every forerunner token to its label token (line 4 in Table 1)
- `"yi_to_sq"`: disconnect the attention link from every label token to the query's forerunner token (line 5 in Table 1)
- `"si_to_si"`: disconnect the attention link from every forerunner token to subsequent forerunner token (Table 2)

In [2]:
# Cell 2: Model and huggingfacetoken configurations

## The huggingface model name to be tested as the LM for ICL. 
## Recommended: "meta-llama/Meta-Llama-3-8B", "tiiuae/falcon-7b", "meta-llama/Meta-Llama-3-70B", "tiiuae/falcon-40b"
ICL_model_name = "tiiuae/falcon-7b" 

## Whether to use the quantized version of the model. 
## This experiment is only available for unquantized models.
quantized = False

## The huggingface token to access the model. If you use the Llama model, you need to set this.
huggingface_token = "your token here"


# Experiment parameters

## The selected token type to calculate the KA. Alternative: "none" (forerunner token s), "label_words" (y), "last_sentence_token" (x).
ICL_selected_token_type = "label_words" 

## The demonstration numbers. Recommended: 0, 1, 2, 4, 8, 12.
k = 4 

## The used dataset index from the StaICC library. Alternative: 0, 1, 2, 3, 4, 5. See the README.md for more information.
dataset_index = 2 

## Force the ICL_model to reload, even the ICL_model is already in the variables. 
## Recommended: False.
model_forced_reload = False

## In which layers (ratios) the ablation is conducted? Start and end layer number percentages.
## Same as the column label in the Table 1.
maskstart = 0
maskend = 0.25

## Ablation type 
ablation_type = "xq_to_sq"

In [ ]:
# Cell 3: Load the data and build the test inputs.

bench = StaICC.Normal(k)
prompts, queries = load_model_and_data.load_data_from_StaICC_experimentor(bench[dataset_index])

In [ ]:
# Cell 4: Load the models.

vars_dict = vars() if "ICL_model" in vars() else locals()
if "ICL_model" not in vars_dict or model_forced_reload:
    ICL_model, ICL_tknz = load_model_and_data.load_ICL_model(ICL_model_name, huggingface_token = huggingface_token, quantized = quantized)
    loaded = True

**Part II: Run the Experiment**

What you should do:

1. Run the Cell 4.

In [ ]:
# Cell 4: Run the ablation study.

## Replace the inference with our method to enable the ablation study.
if "falcon" in ICL_model_name:
    inference_type = "falcon"
    ablation_study.make_falcon_model(ICL_model)
else:
    inference_type = "transformer"
    ablation_study.make_llama_model(ICL_model)

## Run the inference.
exp, ctrl = ablation_study.Masked_ICL_inference(
    prompts,
    ICL_model,
    ICL_tknz, 
    bench[dataset_index],
    inference_type = inference_type,
    mask_start_layer = int(maskstart * ICL_model.config.num_hidden_layers),
    mask_end_layer = int(maskend * ICL_model.config.num_hidden_layers),
    attention_mask_type = ablation_type,
    run_control_experiment_parallelly = True
)

**Part III: Report the Result**

What you should do:

1. Run the Cell 5, and read the results in the echo.

In [ ]:
# Cell 5: Calculate the accuracy.

print("Accuracy of the Ablation Study on dataset: " + str(dataset_index) + ", Model: " + ICL_model_name + ", Ablation Type: " + ablation_type + ", Mask Start: " + str(maskstart) + ", Mask End: " + str(maskend))
bench[dataset_index]._repeat_times = 1
print(bench[dataset_index](input_prediction = exp)[0]['accuracy'])

print("")

print("Accuracy of the CONTROLLED Ablation Study on dataset: " + str(dataset_index) + ", Model: " + ICL_model_name + ", Ablation Type: Random" + ", Mask Start: " + str(maskstart) + ", Mask End: " + str(maskend))
bench[dataset_index]._repeat_times = 1
print(bench[dataset_index](input_prediction = ctrl)[0]['accuracy'])